<a href="https://colab.research.google.com/github/ShinAsakawa/2015corona/blob/master/2023notebooks/2023_0618_1_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
date: 2023_0618
---

stablebaselines3
from <https://araffin.github.io/post/sb3/>

# Stable Baselines3 Tutorial - Getting Started

Github repo: https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3/

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

SB3-Contrib: https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo

[RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) は，Stable Baselines3 を用いた強化学習 (RL) のための訓練フレームワーク。
<!-- [RL Baselines3 Zoo](https://github.com/DLR-RM/rl-baselines3-zoo) is a training framework for Reinforcement Learning (RL), using Stable Baselines3.-->

訓練，エージェントの評価，ハイパーパラメータの調整，結果のプロット，ビデオ撮影のためのスクリプトが用意されている。
<!-- It provides scripts for training, evaluating agents, tuning hyperparameters, plotting results and recording videos. -->


## はじめに <!-- ## Introduction-->

このノートブックでは，stable baselines ライブラリの基本的な使い方，つまり RL モデルの作成方法，訓練方法，評価方法を学ぶ。
すべてのアルゴリズムは同じインターフェイスを共有しているため，あるアルゴリズムから別のアルゴリズムへの切り替えがいかに簡単であるかを確認することができる。
<!--In this notebook, you will learn the basics for using stable baselines library: how to create a RL model, train it and evaluate it.
Because all algorithms share the same interface, we will see how simple it is to switch from one algorithm to another. -->


##  Pip を用いた依存関係と安定した Baselines3 のインストール <!-- ## Install Dependencies and Stable Baselines3 Using Pip-->

完全な依存関係のリストは [README](https://github.com/DLR-RM/stable-baselines3) に記載されている。
<!-- List of full dependencies can be found in the [README](https://github.com/DLR-RM/stable-baselines3). -->


Zsh などを使っている M1 Mac などでは，シングルクォートが必須
```
pip install 'stable-baselines3[extra]'
```

In [ ]:
from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())
if isColab:
    !pip install jupyter-black

In [2]:
# for autoformatting
%load_ext jupyter_black

In [ ]:
if isColab:
    !apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
    !pip install "stable-baselines3[extra]>=2.0.0a4"

## 輸入 Imports

Stable-Baselines3は、[gym interface](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html)に従った環境上で動作する。
利用可能な環境のリストは [こちら](https://gymnasium.farama.org/environments/classic_control/) で見ることができる。
<!-- Stable-Baselines3 works on environments that follow the [gym interface](https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html).
You can find a list of available environment [here](https://gymnasium.farama.org/environments/classic_control/).-->

すべてのアルゴリズムがすべての行動空間で動作するわけではない。
詳しくはこの [総括表](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) 参照。
<!-- Not all algorithms can work with all action spaces, you can find more in this [recap table](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html) -->

In [4]:
import gymnasium as gym
import numpy as np

最初に輸入する必要があるのは RL モデルで，どの問題で何が使えるかはドキュメントを確認せよ。
<!-- The first thing you need to import is the RL model, check the documentation to know what you can use on which problem -->

In [5]:
from stable_baselines3 import PPO

次に輸入する必要があるのは，(方針/価値関数の) ネットワークを作成するために使用される方針クラスである。
コンストラクタで直接文字列を使用することができるので，このステップはオプションである：
<!-- The next thing you need to import is the policy class that will be used to create the networks (for the policy/value functions).
This step is optional as you can directly use strings in the constructor: -->

`PPO(MlpPolicy, env)` の替わりに `PPO('MlpPolicy', env)`

SAC のようないくつかのアルゴリズムは，独自の `MlpPolicy` を持つので，方針に文字列を使うことが推奨される。
<!-- Note that some algorithms like `SAC` have their own `MlpPolicy`, that's why using string for the policy is the recommended option. -->

In [ ]:
from stable_baselines3.ppo.policies import MlpPolicy

## Gym env の作成とエージェントの実体化 <!-- ## Create the Gym env and instantiate the agent-->

この例では，古典的な制御問題である CartPole 環境を使用する。
<!--For this example, we will use CartPole environment, a classic control problem.-->

“ポールは，摩擦のない軌道を移動するカートに，(アクチュエータなしの) ジョイントで取り付けられている。
この系は，カートに +1 または -1 の力を加えることで制御される。
振り子は直立した状態でスタートし，倒れないようにすることが目標となる。
振り子が直立したままであれば，1 タイムステップごとに +1 の報酬が与えられる。"
<!-- "A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track.
The system is controlled by applying a force of +1 or -1 to the cart.
The pendulum starts upright, and the goal is to prevent it from falling over.
A reward of +1 is provided for every timestep that the pole remains upright. " -->

倒立振子 (Cartpole) 環境  <!--environment-->: [https://gymnasium.farama.org/environments/classic_control/cart_pole/](https://gymnasium.farama.org/environments/classic_control/cart_pole/)

<center>
<img src="https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif" width="33%">
</center>

<!-- ![Cartpole](https://cdn-images-1.medium.com/max/1143/1*h4WTQNVIsvMXJTCpXm_TAw.gif) -->

MlpPolicy を選んだのは，倒立振子 (CartPole) 課題の観測が画像ではなく，特徴ベクトルであるためである。
<!-- We chose the MlpPolicy because the observation of the CartPole task is a feature vector, not images.-->

使用する行動の種類 (離散／連続) は，環境の行動空間から自動的に推論される。
<!-- The type of action to use (discrete/continuous) will be automatically deduced from the environment action space -->

ここでは [近位方針最適化 Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) アルゴリズムを使用している。
これは アクター・クリティック (Actor-Critic) 法の一つで，価値関数を使用して方針の勾配降下を (分散を減らすことによって) 改善するアルゴリズムである。
<!-- Here we are using the [Proximal Policy Optimization](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html) algorithm, which is an Actor-Critic method: it uses a value function to improve the policy gradient descent (by reducing the variance). -->

[A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html) (複数のワーカーを持ち，探索にエントロピーボーナスを使う) と [信頼領域方針最適化 TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) (信頼領域を用い，安定性を向上させて性能の壊滅的低下を回避する) のアイデアを組み合わせたものである。
<!-- It combines ideas from [A2C](https://stable-baselines3.readthedocs.io/en/master/modules/a2c.html) (having multiple workers and using an entropy bonus for exploration) and [TRPO](https://stable-baselines.readthedocs.io/en/master/modules/trpo.html) (it uses a trust region to improve stability and avoid catastrophic drops in performance).-->

PPO はオンポリシーアルゴリズムであり，ネットワークの更新に使用する軌跡は最新のポリシーを使用して収集されなければならないことを意味する。
通常，[DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html), [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html) のようなオフポリシーアルゴリズムよりもサンプル効率が劣るが，ウォールクロック時間に関してははるかに高速である。
<!--PPO is an on-policy algorithm, which means that the trajectories used to update the networks must be collected using the latest policy.
It is usually less sample efficient than off-policy alorithms like [DQN](https://stable-baselines.readthedocs.io/en/master/modules/dqn.html), [SAC](https://stable-baselines3.readthedocs.io/en/master/modules/sac.html) or [TD3](https://stable-baselines3.readthedocs.io/en/master/modules/td3.html), but is much faster regarding wall-clock time. -->


In [7]:
env = gym.make("CartPole-v1")

model = PPO(MlpPolicy, env, verbose=0)

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


エージェントを評価するためのヘルパー関数を作成する：
<!-- We create a helper function to evaluate the agent: -->

In [8]:
from stable_baselines3.common.base_class import BaseAlgorithm


def evaluate(
    model: BaseAlgorithm,
    num_episodes: int = 100,
    deterministic: bool = True,
) -> float:
    """
    `num_eposodes` に渡って強化学習エージェントを評価する

    :param model: 強化学習エージェント
    :param env: gym 環境
    :param num_episodes: 評価のためのエピソード回数
    :param deterministic: 決定論的行動を用いるか否かのフラグ
    :return: 直近 `num_episodes` の平均報酬
    """
    # この関数は，単一環境でのみ動作する
    vec_env = model.get_env()
    obs = vec_env.reset()
    all_episode_rewards = []
    for _ in range(num_episodes):
        episode_rewards = []
        done = False
        # 注: SB3 VecEnv は自動的にリセットされる
        # https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecenv-api-vs-gym-api
        # obs = vec_env.reset()
        while not done:
            # _states は LSTM 方針で有益
            # `deterministic` は決定論的行動を行うか否か
            action, _states = model.predict(obs, deterministic=deterministic)
            # `action`, `rewards`, `dones` は配列
            # vectorized env を用いているため
            obs, reward, done, _info = vec_env.step(action)
            episode_rewards.append(reward)

        all_episode_rewards.append(sum(episode_rewards))

    mean_episode_reward = np.mean(all_episode_rewards)
    print(f"Mean reward: {mean_episode_reward:.2f} - Num episodes: {num_episodes}")

    return mean_episode_reward

訓練されていないエージェントを評価してみる。
このために，初期状態はランダムなエージェントである。
<!-- Let's evaluate the un-trained agent, this should be a random agent. -->

In [9]:
# 訓練前の乱数で初期化された動作主
mean_reward_before_train = evaluate(model, num_episodes=100, deterministic=True)

Mean reward: 9.13 - Num episodes: 100


Stable-Baselines はすでにそのヘルパーを提供している：
<!-- Stable-Baselines already provides you with that helper: -->

In [10]:
from stable_baselines3.common.evaluation import evaluate_policy

In [11]:
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100, warn=False)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

mean_reward: 9.17 +/- 0.68


## エージェント (動作主) の訓練と評価

In [12]:
# 動作主を 10000 ステップ訓練
model.learn(total_timesteps=10_000)

In [ ]:
# 訓練済の動作主を評価
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=100)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

訓練はうまくいったようで，平均報酬は大きく伸びた！
<!-- Apparently the training went well, the mean reward increased a lot ! -->

### 動画撮影の準備 <!-- ### Prepare video recording -->

In [13]:
# フェイク画面のセットアップ，そうしないとレンダリングに失敗する
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [14]:
import base64
from pathlib import Path

from IPython import display as ipythondisplay


def show_videos(video_path="", prefix=""):
    """
    https://github.com/eleurent/highway-env より援用

    :param video_path: (str) 動画が格納されているフォルダのパス
    :param prefix: (str) この接頭辞のついた動画のみを表示する
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

[VecVideoRecorder](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) ラッパーを使って動画を録画する。
このラッパーについては，次のノートブックで学ぶ。
<!-- We will record a video using the [VecVideoRecorder](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#vecvideorecorder) wrapper, you will learn about those wrapper in the next notebook. -->

In [15]:
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv


def record_video(env_id, model, video_length=500, prefix="", video_folder="videos/"):
    """
    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

### 学習済みエージェントの可視化
<!-- ### Visualize trained agent -->


In [ ]:
record_video("CartPole-v1", model, video_length=500, prefix="ppo-cartpole")

In [ ]:
show_videos("videos", prefix="ppo")

## ボーナス：1 行で RL モデルを訓練 <!-- ## Bonus: Train a RL Model in One Line -->

使用するポリシークラスが推論され，環境も自動的に作成される。
どちらも [登録済 (registered)](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html) であるため，動作する。
<!-- The policy class to use will be inferred and the environment will be automatically created.
This works because both are [registered](https://stable-baselines3.readthedocs.io/en/master/guide/quickstart.html). -->

In [ ]:
model = PPO('MlpPolicy', "CartPole-v1", verbose=1).learn(1000)

## まとめ <!-- ## Conclusion-->

このノートブックでは
* stablebaselines3 を使って RL モデルを定義，訓練する方法を示した。たった 1 行のコードで可能である ;)

<!-- In this notebook we have seen:
- how to define and train a RL model using stable baselines3, it takes only one line of code ;) -->